In [ ]:
import os,sys
import kfp
import json

### User Variables

In [ ]:
## Fill in this cell with the repo names that you have chosen

code_repo = "mnist"
dataset_repo = "mnist"
model_repo = "mnist"

### Get/Set Environment Variables

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
project_id = os.environ.get("DKUBE_PROJECT_ID")
project_name = os.environ.get("DKUBE_PROJECT_NAME","mnist")
user_name = os.environ.get("DKUBE_USER_LOGIN_NAME")
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"), existing_token=token, namespace=os.getenv("USERNAME"))

### Setup Pipeline Definitions

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def mnist_pipeline(program=code_repo, dataset=dataset_repo, model=model_repo, project_id=project_id):

    train       = dkube_training_op(container='{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0-17"}',
                                    framework="tensorflow", version="2.0.0",
                                    tags=tags,
                                    name=training_run_name,
                                    program=str(program), run_script="python mnist/train.py",
                                    datasets=json.dumps([str(dataset)]), outputs=json.dumps([str(model)]),
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1"}]',
                                    auth_token=token)

    serving     = dkube_serving_op(model=train.outputs['artifact'], device='cpu', 
                                    name=deployment_name,
                                    serving_image='{"image":"ocdr/tensorflowserver:2.0.0"}',
                                    transformer_image='{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0-17"}',
                                    transformer_project=str(program),
                                    transformer_code='mnist/transformer.py', auth_token=token).after(train)

### Create Pipeline, Deployment & Run names using Base and Random digits

In [ ]:
# Create a random set of digits for the names
res = ''.join(random.choices(string.ascii_lowercase + string.digits, k=4))

# Create the deployment, experiment, & run names based on project & user
if project_id:
    tags = json.dumps([f"project:{project_id}"])
    run_name = f"[{project_name}] {user_name}:mnist-pl-%s"%res
    experiment = f"[{project_name}] mnist"
    pipeline_name = f"[{project_name}] mnist-Pipeline.zip"
    training_run_name = f"{project_name}-{user_name}-mnist-pl-%s"%res
    deployment_name = f"{project_name}-{user_name}-mnist-pl-%s"%res
else:
    tags = []
    run_name = f"{user_name}:mnist-pl-%s"%res
    experiment = "default"
    pipeline_name = "mnist-Pipeline.zip"
    training_run_name = f"{user_name}-mnist-pl-%s"%res
    deployment_name = f"{user_name}-mnist-pl-%s"%res

### Create the Pipeline

In [ ]:
# Create the pipeline
client.create_run_from_pipeline_func(mnist_pipeline, run_name=run_name, experiment_name=experiment, arguments={})

### Submit Pipeline

In [ ]:
# Generate & upload pipeline
import kfp.compiler as compiler
compiler.Compiler().compile(mnist_pipeline, pipeline_name)

# Upload Pipeline to DKube if it does not exist
pipeline_id = client.get_pipeline_id(pipeline_name)

if pipeline_id:
    print("Pipeline already exists within DKube, will use existing version")
else:
    client.upload_pipeline(pipeline_name)